In [3]:
import csv
import os
import random
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Verificar existencia de archivos y columnas
def verificar_o_crear_archivo(nombre_archivo, columnas):
    if not os.path.isfile(nombre_archivo):
        with open(nombre_archivo, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(columnas)
        logging.info(f"Archivo {nombre_archivo} creado con columnas: {columnas}")

# Verificar o crear archivos necesarios
verificar_o_crear_archivo("cobertura.csv", ["CodigoPostal", "Cobertura"])
verificar_o_crear_archivo("registros.csv", ["CodigoPostal", "Nombre", "Apellido", "Correo", "Telefono", "TipoVivienda", "Caracteristicas", "Necesidades", "Alarma"])

# Definir la clase ChatbotADT
class ChatbotADT:
    def __init__(self, modelo=None):
        self.nombre = ""
        self.apellido = ""
        self.correo = ""
        self.telefono = ""
        self.codigo_postal = ""
        self.tipo_vivienda = ""
        self.caracteristicas = ""
        self.necesidades = ""
        self.alarma = ""
        self.servicios_adicionales = ""
        self.modelo = modelo

    def iniciar_conversacion(self):
        try:
            logging.info("Iniciando conversación con el usuario.")
            self.nombre = self.pedir_input("¡Hola! Bienvenido al servicio de seguridad de ADT. ¿Cómo te llamas?")
            self.codigo_postal = self.pedir_input(f"Encantado, {self.nombre}. Para poder ayudarte mejor, ¿podrías proporcionarme tu código postal?", codigo_postal=True)
            if self.verificar_cobertura():
                self.recoger_datos_contacto()
            else:
                self.registrar_no_cobertura()
                self.guardar_cobertura(self.codigo_postal, "No")
        except Exception as e:
            logging.error(f"Error en la conversación: {e}")
            print("Ocurrió un error. Por favor, intenta nuevamente más tarde.")

    def pedir_input(self, mensaje, codigo_postal=False, email=False, telefono=False):
        entrada = input(f"{mensaje}\nUsuario: ").strip()
        if codigo_postal:
            while not (entrada.isdigit() and len(entrada) == 5):
                entrada = input("Por favor, introduce un código postal válido de 5 dígitos.\nUsuario: ").strip()
        elif email:
            while "@" not in entrada or "." not in entrada:
                entrada = input("Por favor, introduce una dirección de correo electrónico válida.\nUsuario: ").strip()
        elif telefono:
            while not (entrada.isdigit() and len(entrada) >= 7):
                entrada = input("Por favor, introduce un número de teléfono válido.\nUsuario: ").strip()
        else:
            while not entrada:
                entrada = input("Por favor, introduce una respuesta válida.\nUsuario: ").strip()
        return entrada

    def verificar_cobertura(self):
        logging.info(f"Verificando cobertura para el código postal {self.codigo_postal}.")
        if self.modelo:
            try:
                prediccion = self.modelo.predict([[int(self.codigo_postal)]])
                if prediccion[0] == 1:
                    print("Cobertura disponible según el modelo.")
                    return True
                else:
                    print("No hay cobertura disponible según el modelo.")
                    return False
            except Exception as e:
                logging.error(f"Error al predecir cobertura: {e}")
                return False
        else:
            return self.verificar_cobertura_csv()

    def verificar_cobertura_csv(self):
        try:
            with open("cobertura.csv", mode='r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    if row["CodigoPostal"] == self.codigo_postal:
                        return row["Cobertura"] == "Sí"
            return False
        except FileNotFoundError:
            print("El archivo de cobertura no se encontró. Creando uno nuevo.")
            with open("cobertura.csv", mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["CodigoPostal", "Cobertura"])
            return False
        except csv.Error as e:
            logging.error(f"Error al leer el archivo de cobertura: {e}")
            return False

    def recoger_datos_contacto(self):
        try:
            self.tipo_vivienda = self.pedir_input("¿La consulta es para una vivienda habitual o una segunda residencia?")
            if self.tipo_vivienda.lower() in ["habitual", "1"]:
                self.detalle_vivienda_habitual()
            elif self.tipo_vivienda.lower() in ["segunda residencia", "segunda", "2"]:
                self.detalle_vivienda_segunda()
            else:
                print("Lo siento, no entendí tu respuesta. ¿Podrías repetirla?")
                self.recoger_datos_contacto()
            self.preguntar_contacto()
            self.registrar_en_base_de_datos()
        except Exception as e:
            logging.error(f"Error al recoger datos de contacto: {e}")

    def detalle_vivienda_habitual(self):
        self.necesidades = self.pedir_input("Perfecto, {self.nombre}. ¿Vives solo o con más personas? ¿Tienes niños, animales o alguna persona mayor en casa?")
        self.caracteristicas = self.pedir_input("¿Cuántos accesos principales tiene tu propiedad?")
        self.caracteristicas += ", " + self.pedir_input("¿Tienes jardín o áreas externas importantes?")
        self.caracteristicas += ", " + self.pedir_input("¿Hay objetos de valor específicos que quieras proteger?")
        self.preguntar_sistema_alarma()

    def detalle_vivienda_segunda(self):
        self.caracteristicas = self.pedir_input("Nuestras soluciones son ideales para proteger propiedades mientras no estás. ¿Cuánto tiempo suele estar deshabitada tu segunda residencia durante el año?")
        self.preguntar_sistema_alarma()

    def preguntar_sistema_alarma(self):
        respuesta = self.pedir_input("¿Ya tienes algún sistema de alarma instalado o estás buscando instalar uno nuevo?")
        if "adt" in respuesta.lower():
            print("El número de atención al cliente de ADT es 91 444 44 00. Esta línea es únicamente para consultas sobre contrataciones. ¡Muchas gracias!")
        elif respuesta.lower() in ["sí", "si", "yes", "y"]:
            self.alarma = self.pedir_input("¿Con qué empresa tienes instalado el sistema de alarma?")
        elif respuesta.lower() in ["no", "n"]:
            self.alarma = "No"

    def preguntar_contacto(self):
        self.telefono = self.pedir_input("Para poder contactarte y ofrecerte una oferta personalizada, ¿podrías proporcionar tu número de teléfono?", telefono=True)
        self.correo = self.pedir_input("¿Y tu correo electrónico?", email=True)
        print("Muchas gracias. Nuestro equipo comercial te contactará de lunes a viernes de 9 a 17 horas.")

    def registrar_en_base_de_datos(self):
        datos_registro = [self.codigo_postal, self.nombre, self.apellido, self.correo, self.telefono, self.tipo_vivienda, self.caracteristicas, self.necesidades, self.alarma]
        self.almacenar_registro(datos_registro)
        logging.info(f"Datos registrados para el usuario {self.nombre}.")
        print("Gracias por proporcionar tus datos. Se han registrado correctamente en la base de datos.")
        print("¿Hay algo más en lo que pueda ayudarte?")
        print("Gracias por usar nuestro servicio. Que tenga un buen día.")

    def almacenar_registro(self, datos_registro):
        header = ["CodigoPostal", "Nombre", "Apellido", "Correo", "Telefono", "TipoVivienda", "Caracteristicas", "Necesidades", "Alarma"]
        try:
            file_exists = os.path.isfile("registros.csv")
            with open("registros.csv", mode='a', newline='') as file:
                writer = csv.writer(file)
                if not file_exists:
                    writer.writerow(header)
                writer.writerow(datos_registro)
        except csv.Error as e:
            logging.error(f"Error al escribir en el archivo de registros: {e}")

    def registrar_no_cobertura(self):
        print("Lamentablemente, no ofrecemos servicio en tu área actualmente.")
        print("Si deseas hablar con nuestra empresa, por favor contáctanos en el número de Atención al Cliente de ADT: 91 444 44 00 o envía un correo a acliente@adt.com.es.")
        self.guardar_cobertura(self.codigo_postal, "No")
        logging.info(f"Solicitud de cobertura registrada para el código postal {self.codigo_postal}.")

    def guardar_cobertura(self, codigo_postal, cobertura):
        try:
            file_exists = os.path.isfile("cobertura.csv")
            with open("cobertura.csv", mode='a', newline='') as file:
                writer = csv.writer(file)
                if not file_exists:
                    writer.writerow(["CodigoPostal", "Cobertura"])
                writer.writerow([codigo_postal, cobertura])
        except csv.Error as e:
            logging.error(f"Error al escribir en el archivo de cobertura: {e}")

# Sección de Machine Learning
# Función para cargar y preprocesar datos
def cargar_y_preprocesar_datos(archivo_csv):
    try:
        data = pd.read_csv(archivo_csv)
        data['Cobertura'] = data['Cobertura'].apply(lambda x: 1 if x == 'Sí' else 0)
        data['CodigoPostal'] = data['CodigoPostal'].astype(str).str.extract('(\d+)').astype(int)
        return data
    except pd.errors.EmptyDataError:
        logging.error("El archivo CSV está vacío.")
        return pd.DataFrame(columns=['CodigoPostal', 'Cobertura'])
    except pd.errors.ParserError as e:
        logging.error(f"Error al parsear el archivo CSV: {e}")
        return pd.DataFrame(columns=['CodigoPostal', 'Cobertura'])

# Función para entrenar el modelo de Machine Learning
def entrenar_modelo(data):
    if data.empty:
        logging.warning("Los datos de cobertura están vacíos. No se puede entrenar el modelo.")
        return None
    X = data[['CodigoPostal']]
    y = data['Cobertura']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    logging.info(f"Precisión del modelo: {accuracy * 100:.2f}%")
    return model

# Cargar y preprocesar los datos
archivo_cobertura = "cobertura.csv"
if os.path.exists(archivo_cobertura):
    data = cargar_y_preprocesar_datos(archivo_cobertura)
    # Entrenar el modelo
    modelo_cobertura = entrenar_modelo(data)
else:
    logging.warning(f"El archivo {archivo_cobertura} no existe. No se puede entrenar el modelo.")
    modelo_cobertura = None

# Instanciamos el chatbot y comenzamos la conversación
chatbot = ChatbotADT(modelo=modelo_cobertura)
chatbot.iniciar_conversacion()

# Si el archivo de cobertura se actualiza con nuevos datos, volvemos a entrenar el modelo
def verificar_actualizacion(archivo_csv, modelo_actual):
    data_actualizada = cargar_y_preprocesar_datos(archivo_csv)
    nuevo_modelo = entrenar_modelo(data_actualizada)
    if nuevo_modelo:
        return nuevo_modelo
    return modelo_actual

if os.path.exists(archivo_cobertura):
    modelo_cobertura = verificar_actualizacion(archivo_cobertura, modelo_cobertura)


2024-06-08 15:16:40,191 - INFO - Precisión del modelo: 81.10%
2024-06-08 15:16:40,194 - INFO - Iniciando conversación con el usuario.


¡Hola! Bienvenido al servicio de seguridad de ADT. ¿Cómo te llamas?
Usuario:  paloma
Encantado, paloma. Para poder ayudarte mejor, ¿podrías proporcionarme tu código postal?
Usuario:  41001


2024-06-08 15:17:17,648 - INFO - Verificando cobertura para el código postal 41001.
/Users/pablobasapacheco/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Cobertura disponible según el modelo.


¿La consulta es para una vivienda habitual o una segunda residencia?
Usuario:  segunda residencia
Nuestras soluciones son ideales para proteger propiedades mientras no estás. ¿Cuánto tiempo suele estar deshabitada tu segunda residencia durante el año?
Usuario:  epoca de vacaciones
¿Ya tienes algún sistema de alarma instalado o estás buscando instalar uno nuevo?
Usuario:  no
Para poder contactarte y ofrecerte una oferta personalizada, ¿podrías proporcionar tu número de teléfono?
Usuario:  654321987
¿Y tu correo electrónico?
Usuario:  prueba@gmail.com


2024-06-08 15:17:55,498 - INFO - Datos registrados para el usuario paloma.


Muchas gracias. Nuestro equipo comercial te contactará de lunes a viernes de 9 a 17 horas.
Gracias por proporcionar tus datos. Se han registrado correctamente en la base de datos.
¿Hay algo más en lo que pueda ayudarte?
Gracias por usar nuestro servicio. Que tenga un buen día.


2024-06-08 15:18:13,816 - INFO - Precisión del modelo: 81.10%
